In [1]:
from functions import *
from ase.db import connect
from ase.calculators.singlepoint import SinglePointCalculator
import re
from pathlib import Path

os.makedirs("02-data_db/slab/", exist_ok=True)

#index_tags_path = './index_tags.json'   #if tags is None
current_directory = './01-data/traj/slab' 
extension_to_find = ".traj"
result = find_files_with_extension(current_directory, extension_to_find)
len(result)

db_path = '02-data_db/slab/slab_111.db'
relax_db_path = '02-data_db/slab/slab_111_relax.db'

if os.path.exists(db_path):
    raise Exception('{db} exists. Please delete it before proceeding.')

  
with connect(db_path) as db:
    for file_path in result:
        traj = read(file_path,index=":")
        name = get_filename_from_path(file_path)
        for atoms in traj:
            assert atoms.get_tags() is not None 
            db.write(atoms,name=name)

relax_db_path = get_relax_db(db_path, relax_db_path)

! ase db $relax_db_path 

[1, 2, 3, 4, 5, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
id|age|formula |calculator|  energy|natoms| fmax|pbc|  volume|charge|    mass| smax|magmom
 1| 1s|Al16Cu48|vasp      |-235.448|    64|0.019|TTT|2314.484| 0.000|3481.913|0.012|      
 2| 1s|Cd16Cu48|vasp      |-174.189|    64|0.020|TTT|2547.863| 0.000|4848.832|0.010|      
 3| 1s|Co16Cu48|vasp      |-263.337|    64|0.015|TTT|2194.475| 0.000|3993.139|0.010|23.107
 4| 1s|Ag16Cu48|vasp      |-204.163|    64|0.011|TTT|2446.657| 0.000|4776.099|0.007|      
 5| 1s|Cu48As16|vasp      |-241.987|    64|0.012|TTT|2408.544| 0.000|4248.954|0.017|      
 6| 0s|Au16Cu48|vasp      |-219.968|    64|0.013|TTT|2452.869| 0.000|6201.673|0.010|      
 7| 0s|Cu64    |vasp      |-223.927|    64|0.009|TTT|2228.938| 0.000|4066.944|0.008|      
 8| 0s|Cu48Ge16|vasp      |-239.508|    64|0.015|TTT|2377.444| 0.000|4212.288|0.018|      
 9| 0s|Cu48Ni16|vasp      |-249.230|    64|0.012|TTT|2184.304| 0.000|3989.302|0.011| 0.001
10| 0s|Cu48Pd16|vasp      |

In [ ]:
os.makedirs("./04-Eads", exist_ok=True)
def write_db(directory_traj,ns = 3, ads = True):
    os.makedirs("02-data_db", exist_ok=True)
    db_name = get_filename_from_path(directory_traj[:])+f"_ns{ns}"
    db_path = './02-data_db/' + db_name + '.db'
    ads_db_path = "./02-data_db/ads_" + get_filename_from_path(db_path) +".db"
    if os.path.exists(db_path):
        os.remove(db_path)
    if os.path.exists(ads_db_path):
        os.remove(ads_db_path)
    result = find_files_with_extension(directory_traj, ".traj")
    print(len(result),result[0])

    if "_111" in get_filename_from_path(result[0]):
        db_slab_path = "02-data_db/slab/slab_111_relax.db"
    if "_100" in get_filename_from_path(result[0]):
        db_slab_path = "02-data_db/slab/slab_100_relax.db"

    if os.path.exists(db_path):
        raise Exception('{db} exists. Please delete it before proceeding.')
    with connect(db_path) as db:
        for file_path in result:
            traj = read(file_path,index=":")     
            name = get_filename_from_path(file_path)
            for adslab in traj[:ns]:
                if np.max(np.abs(adslab.get_forces())) > 2:
                    continue
                db.write(adslab,name=name)
    ads_db =  get_ads_db(db_path, db_slab_path, ads_db_path)
    ! ase db $ads_db -c id,name,formula,energy,natoms,fmax --limit 5
    with open(f'./04-Eads/{get_filename_from_path(ads_db)}.txt', 'w') as f:
        sys.stdout = f 
        ! ase db $ads_db  -c id,name,formula,energy,natoms,fmax --limit 10000 

    sys.stdout = sys.__stdout__
    if ads == True:
        os.remove(db_path)
    return ads_db

directory_traj = "01-data/traj/ads-3s" 
ns = 3
ads_db = write_db(directory_traj,ns=ns)

169 01-data/traj/ads-3s\Al1Cu3_111_CHO_13.traj
Al1Cu3_111_CHO_13
Al1Cu3_111_CHO_13
Al1Cu3_111_CHO_13
Al1Cu3_111_CHO_23
Al1Cu3_111_CHO_23
Al1Cu3_111_CHO_23
Al1Cu3_111_CHO_24
Al1Cu3_111_CHO_24
Al1Cu3_111_CHO_24
Al1Cu3_111_CHO_6
Al1Cu3_111_CHO_6
Al1Cu3_111_CHO_6
Al1Cu3_111_COH_13
Al1Cu3_111_COH_13
Al1Cu3_111_COH_13
Al1Cu3_111_COH_16
Al1Cu3_111_COH_16
Al1Cu3_111_COH_16
Al1Cu3_111_CO_0
Al1Cu3_111_CO_0
Al1Cu3_111_CO_11
Al1Cu3_111_CO_11
Al1Cu3_111_CO_11
Al1Cu3_111_CO_14
Al1Cu3_111_CO_14
Al1Cu3_111_CO_16
Al1Cu3_111_CO_16
Al1Cu3_111_CO_16
Cd1Cu3_111_CHO_0
Cd1Cu3_111_CHO_0
Cd1Cu3_111_CHO_0
Cd1Cu3_111_CHO_10
Cd1Cu3_111_CHO_10
Cd1Cu3_111_CHO_10
Cd1Cu3_111_CHO_4
Cd1Cu3_111_CHO_4
Cd1Cu3_111_CHO_4
Cd1Cu3_111_COH_0
Cd1Cu3_111_COH_0
Cd1Cu3_111_COH_17
Cd1Cu3_111_COH_17
Cd1Cu3_111_COH_17
Cd1Cu3_111_CO_0
Cd1Cu3_111_CO_0
Cd1Cu3_111_CO_0
Cd1Cu3_111_CO_10
Cd1Cu3_111_CO_10
Cd1Cu3_111_CO_10
Cd1Cu3_111_CO_24
Cd1Cu3_111_CO_24
Cd1Cu3_111_CO_9
Cd1Cu3_111_CO_9
Cd1Cu3_111_CO_9
Co1Cu3_111_CHO_1
Co1Cu3_111_CHO_1
Co1Cu